In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('prep')
from req_fns import resampling_statistics1d, calculate_aprime, runAnalyses, likelihood_betaBinom
os.chdir('../') 
from scipy.stats import spearmanr

# Load files

In [2]:
#project directory 
base_project_dir= os.getcwd()
project_dir = '../expt1/'
results_dir = project_dir + 'results/'
attndat = pd.read_csv(results_dir + 'expt1_attn_results.csv')
wmdat = pd.read_csv(results_dir + 'expt1_wm_results.csv')

all_subj = np.unique(attndat.subject_num)
nsubj = np.size(all_subj)

# Aggregate behavioral and pupillary results

In [3]:
#behavioral accuracy
acc_nonlure = np.array([ np.mean(attndat.acc[np.logical_and(attndat.freq_trials==1,attndat.subject_num==isubj)]==1) for isubj in range(nsubj)])
acc_lure = np.array([ np.mean(attndat.acc[np.logical_and(attndat.freq_trials==0,attndat.subject_num==isubj)]==1) for isubj in range(nsubj)])
print("accuracy of lure trials, acc =", np.round(np.mean((acc_lure)*100),decimals=2),np.round(np.std((acc_lure)*100)/np.sqrt(nsubj),decimals=2))
print("accuracy of non-lure trials, acc =", np.round(np.mean(acc_nonlure*100),decimals=2),np.round(np.std(acc_nonlure*100)/np.sqrt(nsubj),decimals=2))

#pupil size
s_lure = np.array([ np.mean(attndat.s_post[np.logical_and(np.logical_and(
    attndat.freq_trials==0,attndat.subject_num==isubj),attndat.arf==0)]-attndat.s_bl[np.logical_and(np.logical_and(
    attndat.freq_trials==0,attndat.subject_num==isubj),attndat.arf==0)]) for isubj in range(nsubj)])
s_nonlure = np.array([ np.mean(attndat.s_post[np.logical_and(np.logical_and(
    attndat.freq_trials==1,attndat.subject_num==isubj),attndat.arf==0)]-attndat.s_bl[np.logical_and(np.logical_and(
    attndat.freq_trials==1,attndat.subject_num==isubj),attndat.arf==0)]) for isubj in range(nsubj)])
print("pupil size during lure trials, s =", np.round(np.mean(s_lure),decimals=2),np.round(np.std(s_lure)/np.sqrt(nsubj),decimals=2))
print("pupil size during non-lure trials s =", np.round(np.mean(s_nonlure),decimals=2),np.round(np.std(s_nonlure)/np.sqrt(nsubj),decimals=2))

#pupil size, accurate trials only
s_lure_acc = np.array([ np.mean(attndat.s_post[np.logical_and(np.logical_and(np.logical_and(
    attndat.freq_trials==0,attndat.subject_num==isubj),attndat.arf==0),attndat.acc==1)]-attndat.s_bl[np.logical_and(np.logical_and(np.logical_and(
    attndat.freq_trials==0,attndat.subject_num==isubj),attndat.arf==0),attndat.acc==1)]) for isubj in range(nsubj)])
s_nonlure_acc = np.array([ np.mean(attndat.s_post[np.logical_and(np.logical_and(np.logical_and(
    attndat.freq_trials==1,attndat.subject_num==isubj),attndat.arf==0),attndat.acc==1)]-attndat.s_bl[np.logical_and(np.logical_and(np.logical_and(
    attndat.freq_trials==1,attndat.subject_num==isubj),attndat.arf==0),attndat.acc==1)]) for isubj in range(nsubj)])
print("pupil size during accurate lure trials, s =", np.round(np.mean(s_lure_acc),decimals=2),np.round(np.std(s_lure_acc)/np.sqrt(nsubj),decimals=2))
print("pupil size during accurate non-lure trials, s =", np.round(np.mean(s_nonlure_acc),decimals=2),np.round(np.std(s_nonlure_acc)/np.sqrt(nsubj),decimals=2))

#aprime
aprime = calculate_aprime(acc_nonlure,1-acc_lure)
print("attention sensitivity, A' =", np.round(np.mean(aprime),decimals=2),np.round(np.std(aprime)/np.sqrt(nsubj),decimals=2))

#wm performance
wmacc = np.array([ np.mean(wmdat.wm_perf[wmdat.subject_num==isubj]) for isubj in range(nsubj)])
print("wm performance, m =", np.round(np.mean(wmacc,axis=0),decimals=2),np.round(np.nanstd(wmacc)/np.sqrt(nsubj),decimals=2))

#pupil size, retention intervla
s_ret = np.array([ np.nanmean(wmdat.s_ret[np.logical_and(wmdat.subject_num==isubj,wmdat.arf==0)]-
                           wmdat.s_prevstim_bl[np.logical_and(wmdat.subject_num==isubj,wmdat.arf==0)]) for isubj in range(nsubj)])
print("pupil size during retention interval, s =", np.round(np.nanmean(s_ret),decimals=2),np.round(np.std(s_ret)/np.sqrt(nsubj),decimals=2))

accuracy of lure trials, acc = 56.86 3.04
accuracy of non-lure trials, acc = 95.59 0.85
pupil size during lure trials, s = 29.36 4.6
pupil size during non-lure trials s = -0.03 3.42
pupil size during accurate lure trials, s = 26.2 4.97
pupil size during accurate non-lure trials, s = -0.51 3.43
attention sensitivity, A' = 0.86 0.01
wm performance, m = 2.24 0.11
pupil size during retention interval, s = 49.55 8.12


# Computational model

In [4]:
alpha_vals = np.arange(.01,10,.01)
kmax_vals = np.arange(1,7,dtype=int)
alpha = np.empty(nsubj)
kmax = np.empty(nsubj)
for isubj in range(nsubj):
    alpha[isubj], kmax[isubj], _, _, _, _ = runAnalyses(wmdat.wm_perf[wmdat.subject_num==isubj], 
                                                        likelihood_betaBinom, alpha_vals, kmax_vals)

print("alpha: ", np.round(np.mean(alpha),decimals=2),"+/-",np.round(np.std(alpha)/np.sqrt(nsubj),decimals=2))
print("kmax: ", np.round(np.mean(kmax),decimals=2),"+/-",np.round(np.std(kmax)/np.sqrt(nsubj),decimals=2))

alpha:  1.81 +/- 0.23
kmax:  2.83 +/- 0.12


# Individual differences

In [5]:
#correlate working memory and pupil size
r,p = spearmanr(wmacc,s_ret)
print('corr, wmacc pupil size during retention, r= ', np.round(r,decimals=2), 'p=', np.round(p,decimals=4))

corr, wmacc pupil size during retention, r=  0.62 p= 0.0003


# RT before lapses vs non lapses


In [6]:
rttrail_inacc = np.array([ np.mean(attndat.rt_trail[np.logical_and(np.logical_and(
    attndat.freq_trials==0,attndat.subject_num==isubj),attndat.acc!=1)])*1000 for isubj in range(nsubj)])
rttrail_acc = np.array([ np.mean(attndat.rt_trail[np.logical_and(np.logical_and(
    attndat.freq_trials==0,attndat.subject_num==isubj),attndat.acc==1)])*1000 for isubj in range(nsubj)])

print("RT before inaccurate:", np.round(np.mean(rttrail_inacc)),"+/-",np.round(np.std(rttrail_inacc)))
print("RT before accurate:",np.round(np.mean(rttrail_acc)),"+/-",np.round(np.std(rttrail_acc)))

RT before inaccurate: 324.0 +/- 40.0
RT before accurate: 444.0 +/- 48.0


# Behavioral real time triggering

In [7]:
#RT
rt_trail_low = np.array([ np.mean(wmdat.rt_trail[((np.logical_and(
    wmdat.triggered_low==1,wmdat.subject_num==isubj)))])*1000 for isubj in range(nsubj)])
rt_trail_high = np.array([ np.mean(wmdat.rt_trail[((np.logical_and(
    wmdat.triggered_high==1,wmdat.subject_num==isubj)))])*1000 for isubj in range(nsubj)])
print("RT low",np.round(np.mean(rt_trail_low),decimals=0),"+/-",np.round(np.std(rt_trail_low)/np.sqrt(nsubj),decimals=0))
print("RT high", np.round(np.mean(rt_trail_high),decimals=0),"+/-",np.round(np.std(rt_trail_high)/np.sqrt(nsubj),decimals=0))

RT fast 240.0 +/- 8.0
RT slow 519.0 +/- 13.0


In [8]:
#working memory
m_low = np.array([ np.mean(wmdat.wm_perf[((np.logical_and(
    wmdat.triggered_low==1,wmdat.subject_num==isubj)))]) for isubj in range(nsubj)])
m_high = np.array([ np.mean(wmdat.wm_perf[((np.logical_and(
    wmdat.triggered_high==1,wmdat.subject_num==isubj)))]) for isubj in range(nsubj)])
print("m low", np.round(np.mean(m_low),decimals=2),"+/-",np.round(np.std(m_low)/np.sqrt(nsubj),decimals=2))
print("m high", np.round(np.mean(m_high),decimals=2),"+/-",np.round(np.std(m_high)/np.sqrt(nsubj),decimals=2))
p,_ = resampling_statistics1d(m_high-m_low,0)
print("p =",np.round(p,decimals=2))

m low 2.2 +/- 0.1
m high 2.29 +/- 0.11
p = 0.03


In [9]:
#pupil size
strail_m = np.array([ np.mean(wmdat.s_prevstim[np.logical_and(
    wmdat.subject_num==isubj,np.logical_or(wmdat.triggered_high==1,wmdat.triggered_low==1))]) for isubj in range(nsubj)])
strail_high = np.array([ np.mean(wmdat.s_prevstim[(np.logical_and(
    wmdat.subject_num==isubj,wmdat.triggered_high==1))]) for isubj in range(nsubj)])
strail_low = np.array([ np.mean(wmdat.s_prevstim[(np.logical_and(
    wmdat.subject_num==isubj,wmdat.triggered_low==1))]) for isubj in range(nsubj)])
print("s trail low", np.round(np.mean(strail_low-strail_m,axis=0),decimals=2),"+/-",
      np.round(np.std(strail_low-strail_m,axis=0)/np.sqrt(nsubj),decimals=2))
print("s trail high", np.round(np.mean(strail_high-strail_m,axis=0),decimals=2),"+/-",
      np.round(np.std(strail_high-strail_m,axis=0)/np.sqrt(nsubj),decimals=2))
p,_ = resampling_statistics1d(strail_high-strail_low,0)
print("p =",np.round(p,decimals=2))

s trail low -9.12 +/- 5.22
s trail high 9.05 +/- 5.45
p = 0.04


In [10]:
#pupil size, after artifact rejecting
strail_m = np.array([ np.mean(wmdat.s_prevstim[np.logical_and(np.logical_and(
    wmdat.subject_num==isubj,np.logical_or(wmdat.triggered_high==1,
    wmdat.triggered_low==1)),wmdat.arf==0)]) for isubj in range(nsubj)])
strail_high = np.array([ np.mean(wmdat.s_prevstim[np.logical_and(np.logical_and(
    wmdat.subject_num==isubj,wmdat.triggered_high==1),wmdat.arf==0)]) for isubj in range(nsubj)])
strail_low = np.array([ np.mean(wmdat.s_prevstim[np.logical_and(np.logical_and(
    wmdat.subject_num==isubj,wmdat.triggered_low==1),wmdat.arf==0)]) for isubj in range(nsubj)])
print("s trail low", np.round(np.mean(strail_low-strail_m,axis=0),decimals=2),"+/-",
      np.round(np.std(strail_low-strail_m,axis=0)/np.sqrt(nsubj),decimals=2))
print("s trail high", np.round(np.mean(strail_high-strail_m,axis=0),decimals=2),"+/-",
      np.round(np.std(strail_high-strail_m,axis=0)/np.sqrt(nsubj),decimals=2))
p,_ = resampling_statistics1d(strail_high-strail_low,0)
print("p =",np.round(p,decimals=3))

s trail low -13.96 +/- 5.67
s trail high 14.67 +/- 6.41
p = 0.008


In [11]:
os.getcwd()

'/Users/megan/Dropbox/writing/articles/2020_pupil/KeenedeBettencourt_2021/scripts'